In [10]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt 

def imshow(title ="image",image = None, size= 10 ):
    w, h =image.shape[0],image.shape[1]
    aspect_ratio =  w/h
    plt.figure(figsize=(size*aspect_ratio, size))
    plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    plt.show()


## **The Lucas-Kanade Optical Flow Algorithm**

Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movemement of object or camera. It is 2D vector field where each vector is a displacement vector showing the movement of points from first frame to second. Consider the image below (Image Courtesy: Wikipedia article on Optical Flow).
Optical Flow algoritması, bir görüntü dizisindeki hareketli nesnelerin hız vektörlerini hesaplamak için kullanılan bir tekniktir. Bu, her pikselin hareket yönü ve hızının hesaplanmasını sağlar ve bu bilgi genellikle nesne takibi, hareket analizi ve video stabilizasyonu gibi uygulamalarda kullanılır.

Optical Flow algoritmaları, genellikle iki ana yaklaşıma ayrılır: sparse (seyrek) ve dense (yoğun).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Optical_flow_example_v2.png/440px-Optical_flow_example_v2.png)

It shows a ball moving in 5 consecutive frames. The arrow shows its displacement vector. Optical flow has many applications in areas like :

- Structure from Motion
- Video Compression
- Video Stabilization

Optical flow works on several assumptions:

- The pixel intensities of an object do not change between consecutive frames.
- Neighbouring pixels have similar motion.

Learn more - https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html

In [11]:
import cv2
import numpy as np
cap = cv2.VideoCapture(r'C:\Users\cengh\Desktop\ComputerVison\OpticalFlow\walking.avi')
# get  the height and weight frame(integer)

# get the height and width of the frame (integer)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#define codec and create a videowriter object the output avi file
out = cv2.VideoWriter('optical_flow_walking.avi',cv2.VideoWriter_fourcc('M','J','P','G',),30,(width, height))
# for corner detection
feature_params =dict(maxCorners =100,
                     qualityLevel = 0.3,
                     minDistance = 7,
                     blockSize =7)

#Set parameters  for lucas kanade optical algorithm kuantum algorithm
lucas_kenade_params = dict(winSize =(15,15),
                           maxLevel=2,
                           criteria =(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))
# create random colors
color = np.random.randint(0,255,(100,3))

ret , prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

#Find initial corner locations

prev_corners = cv2.goodFeaturesToTrack(prev_gray,mask=None,**feature_params) 

mask =np.zeros_like(prev_frame)
while(True):
    ret , frame =cap.read()
    if ret == True:
        frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        #calculate optimal flow 
        new_corners,status , errors = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prev_corners,None,**lucas_kenade_params)
    good_new = new_corners[status==1]
    good_old = prev_corners[status==1]

#Draw tracks
    for i,(new,old)in enumerate(zip(good_new,good_old)):
        a, b = new.ravel()
        c, d = old.ravel() 
        mask = cv2.line(mask,(a,b),(c,d), color[i].tolist(),2)
        frame = cv2.line(frame,(a,b),5, color[i].tolist(),-1)
        img = cv2.add(frame,mask)
        out.write(img)
        prev_gray =frame_gray.copy()
        prev_corners =good_new.reshape(-1,1,2)
    else:
        break
cap.release()
out.release()


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
